# Chart Recognition

In [1]:
import os
import keras
import pandas
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.models import load_model


Using TensorFlow backend.


In [2]:
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="float32" )
    return data

data_dir = 'charts_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'


In [4]:
img_width, img_height = 224, 224
batch_size = 5
epochs = 10

train_datagen = ImageDataGenerator(
    rescale = 1./224)

test_datagen = ImageDataGenerator(
    rescale = 1./224)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size, 
    class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
    valid_dir,
    target_size = (img_height, img_width),
    class_mode = "categorical")

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height, img_width),
    class_mode = "categorical")

Found 144 images belonging to 6 classes.
Found 43 images belonging to 6 classes.
Found 64 images belonging to 6 classes.


### Fine-tune Model

In [5]:
vgg_model = applications.VGG19(weights = "imagenet", 
                           include_top=False, 
                           input_shape = (img_width, img_height, 3))
for layer in vgg_model.layers[:5]:
    layer.trainable = False
x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(6, activation='softmax')(x)
model_final = Model(inputs=vgg_model.input, outputs=predictions)
for layer in vgg_model.layers:
    layer.trainable = False
model_final.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [6]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs = 8,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples/validation_generator.batch_size)

Epoch 1/8
28/28 [==============================] - 94s - loss: 1.2474 - val_loss: 0.6858
Epoch 2/8
28/28 [==============================] - 78s - loss: 0.4997 - val_loss: 0.6079
Epoch 3/8
28/28 [==============================] - 90s - loss: 0.3232 - val_loss: 0.3809
Epoch 4/8
28/28 [==============================] - 78s - loss: 0.2125 - val_loss: 0.1073
Epoch 5/8
28/28 [==============================] - 93s - loss: 0.1222 - val_loss: 0.1346
Epoch 6/8
28/28 [==============================] - 84s - loss: 0.0876 - val_loss: 0.1606
Epoch 7/8
28/28 [==============================] - 92s - loss: 0.0720 - val_loss: 0.0568
Epoch 8/8
28/28 [==============================] - 78s - loss: 0.0810 - val_loss: 0.0451


In [7]:
checkpoint = ModelCheckpoint("chart_vgg19_wt.h5", 
                             monitor='val_acc', verbose=2, 
                             save_best_only=True, save_weights_only=False, 
                             mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, patience=3, 
                      verbose=2, mode='auto')
layer_num = len(model_final.layers)
for layer in model_final.layers[:21]:
    layer.trainable = False

for layer in model_final.layers[21:]:
    layer.trainable = True
model_final.compile(optimizer=optimizers.Adam(lr=0.0001), 
              loss='categorical_crossentropy', metrics=['accuracy'])
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples/train_generator.batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples/validation_generator.batch_size,
    callbacks = [checkpoint, early])

Epoch 1/10
28/28 [==============================] - 91s - loss: 0.0395 - acc: 1.0000 - val_loss: 0.0512 - val_acc: 1.0000
Epoch 2/10
28/28 [==============================] - 78s - loss: 0.0275 - acc: 1.0000 - val_loss: 0.0130 - val_acc: 1.0000
Epoch 3/10
28/28 [==============================] - 92s - loss: 0.0296 - acc: 1.0000 - val_loss: 0.0404 - val_acc: 1.0000
Epoch 4/10
28/28 [==============================] - 79s - loss: 0.0277 - acc: 1.0000 - val_loss: 0.0695 - val_acc: 1.0000
Epoch 5/10
28/28 [==============================] - 91s - loss: 0.0154 - acc: 1.0000 - val_loss: 0.0483 - val_acc: 1.0000
Epoch 00004: early stopping


In [8]:
model_final.save('chart_vgg19_model.h5')